## Tavily Search
- Tavily Search API is a search engine for AI agents
- [More details](https://python.langchain.com/docs/integrations/tools/tavily_search/)

### Prerequisite
- Install package : `pip install -qU "langchain-community>=0.2.11" tavily-python`

In [13]:
## Read Tavily Key from env file
import os
from dotenv import load_dotenv, find_dotenv
# Load local env file
_ = load_dotenv(find_dotenv())
tavily_api_key = os.environ['TAVILY_API_KEY']

In [14]:
# Open AI Basic Setup
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

In [15]:
## Instantiation
from langchain_community.tools import TavilySearchResults
search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

In [16]:
# Invoke search
search_tool.invoke({"query": "What are the key news headlines in USA related to Stock market on November 1, 2024?"})


[{'url': 'https://apnews.com/article/financial-markets-stocks-dow-nasdaq-a109be4b6056b8eef30daf5728b45f83',
  'content': 'How major US stock indexes fared Wednesday, 11/1/2023 | AP News AP Votecast AP Top 25 Mideast Wars Israel-Hamas War Russia-Ukraine War Global elections ESPAÑOL China Asia Pacific Latin America AP Votecast Results Advance vote Explaining 2024 AP Votecast AP Top 25 AP Top 25 NFL MLB NBA NHL Men’s College Basketball Women’s College Basketball Auto Racing Movie reviews What to Stream Television Book reviews Music Celebrity Interviews Space Animals The Ancient World Climate Medicine More From AP News AP’s Role in Elections AP Leads AP Stylebook About AP News Values and Principles AP’s Role in Elections AP Leads AP Definitive Source Blog AP Images Spotlight Blog AP Stylebook More From AP News AP’s Role in Elections AP Leads AP Stylebook'},
 {'url': 'https://www.bloomberg.com/news/articles/2024-10-31/stock-market-today-dow-s-p-live-updates',
  'content': 'Stock Market Toda

## Chain Search call with LLM Chat 

In [22]:
# Bind with LLM using bind_tools
import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

current_date = datetime.datetime.today()
print(f"current_date is : {current_date}")
prompt = ChatPromptTemplate(
    [
        ("system",f"You are a helpful assistant who can answer to user input in layman English sentences. The date today is {current_date}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}")   
    ]
)

# Bind search tool to llm
model_with_search = model.bind_tools([search_tool])
model_chain = prompt | model_with_search

@chain
def search_tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    model_msg = model_chain.invoke(input_, config=config)
    search_tool_msg = search_tool.batch(model_msg.tool_calls, config=config)
    return model_chain.invoke({**input_, "messages": [model_msg, *search_tool_msg]},config=config)


current_date is : 2024-11-02 09:08:14.780147


In [ ]:
# Invoke search by calling search_tool_chain method
search_tool_chain.invoke("Where did Trump campagin yesterday ? and where is he planning to campaign today ?")

AIMessage(content='Today is November 2, 2024.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 137, 'total_tokens': 148, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-130e4fd1-a7bf-4c85-8794-a7e58b3f6842-0', usage_metadata={'input_tokens': 137, 'output_tokens': 11, 'total_tokens': 148, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})